In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from scrape_utils import WebScrapingUtilities
%run ../load_magic/storage.py
from ha_utils import HeaderAnalysis
from cypher_utils import CypherUtilities

wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
s = Storage()
ha = HeaderAnalysis()
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

CPU times: total: 2.16 s
Wall time: 2.89 s



----
# TASK_SCOPE_HEADERS_LIST

In [5]:

cypher_str = f'''
    MATCH (pos:PartsOfSpeech {{pos_explanation: "Task Scope Header"}}), (np:NavigableParents)
    WHERE np.navigable_parent IN {TASK_SCOPE_HEADERS_LIST}
    MERGE (pos)-[r: SUMMARIZES]->(np);'''
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)

In [ ]:

from datetime import datetime
from urllib.parse import urlparse, parse_qs
%run ../load_magic/dataframes.py
from html_analysis import ElementAnalysis

ea = ElementAnalysis()

In [10]:

def generate_child_strs(verbose=False):
    cypher_str = '''
        MATCH (np:NavigableParents)
        WHERE NOT EXISTS {
            MATCH (:PartsOfSpeech)-[:SUMMARIZES]->(np)
        }
        RETURN np.navigable_parent AS navigable_parent;'''
    row_objs_list = cu.get_execution_results(cypher_str, verbose=verbose)
    if verbose:
        print(len(row_objs_list))
    for row_obj in row_objs_list:
        if 'navigable_parent' in row_obj:
            yield row_obj['navigable_parent']

In [9]:

CHILD_STRS_LIST = list(generate_child_strs(verbose=False))

In [11]:

ZMQInteractiveShell_obj = get_ipython()
if s.pickle_exists('TASK_SCOPE_HEADERS_LIST'):
    TASK_SCOPE_HEADERS_LIST = s.load_object('TASK_SCOPE_HEADERS_LIST')
else:
    TASK_SCOPE_HEADERS_LIST = []
print(len(TASK_SCOPE_HEADERS_LIST))
TASK_SCOPE_HEADERS_DICT = {k: True for k in TASK_SCOPE_HEADERS_LIST}
def get_dictionary_code():
    output_str = ''
    tag_str = CHILD_STRS_LIST.pop()
    while tag_str in TASK_SCOPE_HEADERS_DICT:
        tag_str = CHILD_STRS_LIST.pop()
    output_str += f'\n# {len(CHILD_STRS_LIST):,} to go\n'
    if "'" in tag_str:
        tag_str = tag_str.replace('"', '\\"')
        output_str += f'tag_str = "{tag_str}"\n'
    else:
        output_str += f"tag_str = '{tag_str}'\n"
    output_str += 'TASK_SCOPE_HEADERS_DICT[tag_str] = False\n'
    output_str += 'print(len(TASK_SCOPE_HEADERS_DICT.keys()))\n'
    output_str += 's.store_objects(TASK_SCOPE_HEADERS_DICT=TASK_SCOPE_HEADERS_DICT)\n'
    output_str += 'navigable_parent = cu.clean_text(tag_str)\n'
    output_str += 'if TASK_SCOPE_HEADERS_DICT[tag_str]:\n'
    output_str += '    cypher_str = cu.set_is_task_scope1_cypher_str.format(navigable_parent)\n'
    output_str += 'print(cypher_str)\n'
    output_str += 'with cu.driver.session() as session:\n'
    output_str += '    session.write_transaction(cu.do_cypher_tx, cypher_str)'
    
    return output_str

64


In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_dictionary_code(), replace=True)

In [12]:

# 33,287 to go
tag_str = 'â€<â€<'
TASK_SCOPE_HEADERS_DICT[tag_str] = False
print(len(TASK_SCOPE_HEADERS_DICT.keys()))
s.store_objects(TASK_SCOPE_HEADERS_DICT=TASK_SCOPE_HEADERS_DICT)
navigable_parent = cu.clean_text(tag_str)
if TASK_SCOPE_HEADERS_DICT[tag_str]:
    cypher_str = cu.set_is_task_scope1_cypher_str.format(navigable_parent)
print(cypher_str)
with cu.driver.session() as session:
    session.write_transaction(cu.do_cypher_tx, cypher_str)